http://vision.stanford.edu/aditya86/ImageNetDogs/


In [13]:
import torch
import torchvision
from torchvision import transforms
import os

In [ ]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load the data
train_data = torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/data/train/", transform=transform)
test_data = torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/data/test/", transform=transform)

# Define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)

In [14]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet152

In [15]:
# Define the model
model = resnet152(pretrained=True)

In [18]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Move the model to the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [19]:
# Define the number of epochs
num_epochs = 5

# Train the model
for epoch in range(num_epochs):
    # Train the model on the training set
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Move the data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update the training loss
        train_loss += loss.item() * inputs.size(0)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    test_acc = 0.0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            # Move the data to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Update the test loss and accuracy
            test_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            test_acc += torch.sum(preds == labels.data)

    # Print the training and test loss and accuracy
    train_loss /= len(train_data)
    test_loss /= len(test_data)
    test_acc = test_acc.double() / len(test_data)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Train Loss: {train_loss:.4f} Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")

Epoch [1/5] Train Loss: 2.1092 Test Loss: 1.8212 Test Acc: 0.4978
Epoch [2/5] Train Loss: 1.1464 Test Loss: 1.7331 Test Acc: 0.5526
Epoch [3/5] Train Loss: 0.7720 Test Loss: 1.6210 Test Acc: 0.5915
Epoch [4/5] Train Loss: 0.5512 Test Loss: 1.4654 Test Acc: 0.6331
Epoch [5/5] Train Loss: 0.3804 Test Loss: 1.3835 Test Acc: 0.6600


In [21]:
model_dir = '/content/drive/MyDrive/models/'
os.makedirs(os.path.dirname(model_dir), exist_ok=True)
filename = 'naiive_model.pt'

# Save the entire model
torch.save(model, model_dir+filename)